Heap Data Structure:
 - A complete binary tree that satisfies the heap property, sometimes referred to as a PRIORITY QUEUE. Not to be confused with MONOTONIC QUEUE (see sliding window maximum).
 - Complete Binary Tree: every level, except for the leaves, is completely filled & all of the nodes in the last level are filled from the left.
 - Heap Property: For every node, the value of its children is less than or equal to its own value (max heap) or vice versa (min heap)
    - Therefore, the smallest or largest element will always be at the root of the tree.
- Applications: Used for implementing a priority queue, and heapsort.

Heap Methods:
 - Heapify (building a max heap FROM SCRATCH): O(n)
 - Insert (Push) O(logN)
 - Delete        O(logN)
 - Pop Min / Max O(logN)
 - Peek O(1)

Why?
 - Insert, Delete, Pop may cause the heap to no longer have its heap property, so the tree needs to be repaired. Because the tree has already been built, worst case some nodes may need to be swapped all the way to the bottom, which would require O(logN) calls to our heapify proportional to the height of the tree.
 - Peek is O(1) because the root node, which will be the maximum or minimum value, will be at the front of the array and we can just access it via indexing.
 - Building the heap is O(n), which is counter-intuitive at first. We might think it should be proportional to O(NlogN) because we'll have to go through each node and it will take O(logN) worst-case to find its correct location. In reality, since we're building the heap from scratch, we're starting at floor(N/2) and working our way to index 0 in our array: note that floor(N/2 + 1) : N are all leaf nodes (roughly half of a complete binary tree will be leaf nodes). Since we build the tree at the first parents of the leaf nodes and working our way up, swaps for the MOST PART will be O(1) time. In other words, most nodes will not have to move much because they'll already be at the bottom of the tree and will be swapping between adjacent nodes. Only the very few nodes at the top of the tree might need O(logN) but the operation gets dominated by the lower nodes. Meanwhile, insert and delete are O(logN) because they generally impact the top of the tree which means nodes may have to be swapped all the way down to find its correct location.

How do we implement Heapify, the operation behind insert / delete, and Build Max Heap?
 - Build max heap is called heapify in Python, which can cause some confusion. Think of build max heap as a wrapper to make multiple calls to heapify to correctly place each node in the right location as we traverse our array.
 - Note: for a given index, to access it's left and right child, left child will always be at 2*i if it exists, and the right child will be at 2*i + 1 if it exists. 

In [ ]:
#NOTE: This is a max_heap example, for the min heap our bounds would be flipped.
class Solution:
    def build_max_heap(arr):
        heap_size = len(arr)

        for i in range(heap_size // 2, -1, -1):
            max_heapify(arr, heap_size, i)
    
    def max_heapify(arr, heap_size, i):
        l = 2*i
        r = 2*i + 1
        largest = i

        #we're finding to see if any of the children are larger, and also to find the largest among them.
        if l < heap_size and arr[l] > arr[i]:
            largest = l
        
        if r < heap_size and arr[r] > arr[largest]:
            largest = r
        
        if i != largest:
            #swap i with the largest we found, and continue from where i is now located to repair that side of the heap.
            arr[i], arr[largest] = arr[largest], arr[i]
            max_heapify(arr, heap_size, largest)

Kth Largest Element in Array:
 - This will expose us to many different heap methods!
 - Strategy 1: Create min-heap (default in Python) of size N, pop first N-K elements to be left with k largest element
 - Strategy 2: Use the builtin heapq.nlargest to achieve the same result as strategy 1.

Time Complexity for Strategies 1 and 2? O(nlogn + (n-k)logn + logn)
 - nlogn for n heappush operations (using heapify instead will be O(n) but this is for learning purposes)
 - (n-k)logn for n-k heappop operations
 - logn for last heappop to get kth element.
 - Overall O(nlogn) which is the slowest.

Memory complexity O(n) because we have an n min-heap.

 - Strategy 3 (Best): to save memory, maintain a k min-heap and update it whenever we encounter an element bigger than the min, ensuring the heap will have the k biggest elements.
 - We'll use heapify to sort the first k elements, which will be O(k) time complexity because heapify operates building a tree from the ground up while the other operations repair an established tree from the top down which is more expensive.
 - We'll be using heapq.heapreplace in order to FIRST POP the min and then push our next element O(logk).
 
 Time Complexity: O(nlogk) - although we still go through n elements (technically (n-k) heap-wise, but n dominates here because k we'll generally assume to be smaller), our heap is smaller so less elements we have to traverse when repairing the heap.
 Memory Complexity: O(k) - only k elements in min-heap



In [ ]:
#Strategy 1:
class Solution:
    def kthLargest(self, ums: List[int], k: int) -> int:
        heap = []
        for n in nums:
            heapq.heappush(heap, n)
        
        for _ in range(len(nums)-k):
            heapq.heappop(heap)
        
        return heapq.heapop()


In [ ]:
#Strategy 2:
class Solution:
    def kthLargest(self, nums: List[int], k: int) -> int:
        #returns list of k largest elements, where last one will be smallest.
        return heapq.nlargest(k, nums)[-1] 

In [ ]:
#Strategy 3:
class Solution:
    def kthLargest(self, nums: List[int], k: int) -> int:
        heap = nums[:k]

        heapq.heapify(heap) #O(n) 

        for idx in range(k, len(nums)): #n-k elements:
            #if greater than root (min), we'll replace the min with the greater to maintain heap of k greatest elements
            if nums[idx] > heap[0]:
                heapq.heapreplace(heap, nums[idx])  #O(logk)
        
        return heap[0] #top of heap will be min or kth largest element.

K Closest Points to Origin:
 - In this problem we're exposed to how we can pass in tuples into heaps for tagging and tracking purposes as the heap shifts our elements around.
 - In Python, the built in heap will only build the heap off of the first element in a tuple or array.
 - Generally use a tuple here because we don't want the data to change (immutable), so it's safer.
 - Here, we'll use heapq.heappushpop() which does the opposite of heapq.heapreplace: we first insert our element, allow the heap to repair, then pop the min.
 - Since we want the K smallest distances, we'll have to negate our distances when they go into the heap, so the root of the heap will be the smallest value but greatest absolute distance.
 - Time complexity will be O(nlogk) worst case (here we start with empty heap and either push or pushpop into it if we've already reached the max size we want the heap to be (k)) and memory will be O(k) again. 
 - Note how we use some useful concepts like tuples, heap methods, list comprehension, enumerating objects, and destructuring in this problem to deliver a clean solution!

In [ ]:
class Solution:
    def kClosestPoints(self, points: List[List[int]], k: int) -> List[List[int]]:
        heap = []

        for (x,y) in points:
            dist = -1*(x*x + y*y)**0.5
            #pushpop because heap full, maintain k closest points.
            #we'll push a tuple to keep track of the x,y coordinates that gave us this distance, but only distance (first element) will be used to maintain the heap.
            #only dist will be used in the comparisons. Since we're negative, the most negative (min) but absolute greatest distance will continue getting popped off until we have the k smallest distances (but greatest negatives)
            if len(heap) == k:
                heapq.heappushpop(heap, (dist, x, y)) #O(2logk), one for push, one for pop, but O(logk) simplified because constant time doesn't contribute to the overall complexity. 
            else:
                heapq.heappush(heap, (dist, x, y)) #O(logk)
        
        #return the k closest (but greatest negative) points that we maintained in the heap.
        return [(x,y) for (dist,x,y) in heap]

Top K Frequent Elements:
 - We're interested in grabbing the K most frequent elements from an array, where K is passed in dynamically.
 - To solve via heaps, we'll still use a hashMap to aggregate frequencies for each number O(n)
 - There's a useful collection methods called collections.Counter that can do this for us in one line to obtain {1: freq1, 2: freq2, ...}
 - We'll store the frequencies in an array of tuples (freq, n) [(freq1, 1), (freq2, 2), ...]
   - Why not negate? Since we want the top K frequent elements and the heap is a min-heap by default, we'll need to know what the true min the heap currently is so we can replace it if something bigger comes along. In other words, a min heap is good for aggregating the biggest elements since our comparison (entrypoint) will be at the root which will always be the minimum, so we can replace it if we find something bigger, ensuring we'll only have the biggest.
 - We'll then grab the first k tuples and build a hashmap using the first element in the tuple freq O(k)
 - Keeping the heap the minimum size it needs to be (size k) is important because it will reduce time complexity when we do operations on the heap since the height of the heap will be smaller and traversal will be less expensive.
 - Go through the remainder of the tuples and update the heap so we're storing the K most frequent elements in it.
 - Use list comprehension to grab all of the numbers on the heap and return them.

 - Time complexity: O(nlogk) because worst case the frequencies happen to be sorted, so we'll be performing O(logk) for each frequency (n) we have to go through. However, this is better than O(nlogn) which would've been the case if we made a heap of n elements.
 - Memory complexity: O(k + 2n): O(k) for the heap and O(n) for the tuples worst case, assuming each number in the array is unique, therefore the frequencies array will be the same length as the original.

In [ ]:
class Solution:
    def topKFrequentElements(self, nums: List[int], k: int) -> List[int]:
        freqMap = collections.Counter(nums)
        freqTuples = []
        for num, freq in freqMap.items():
            freqTuples.append((freq, num))
        
        heap = freqTuples[:k]
        heapq.heapify(heap) #O(k)

        for i in range(k, len(freqTuples)):
            tup = freqTuples[i]

            #we found something bigger than the smallest currently in the heap, replace because we want the K biggest!
            if tup[0] > heap[0][0]:
                #Pop, then push
                heapq.heapreplace(heap, tup)  #O(logk), we do it (n-k)logk times
        
        #now that heap as our k biggest elements:
        return [h[1] for h in heap]

Task Scheduler
 - A great and interesting problem that shows you how tasks might get organized and processed by a real Operating System (OS) Scheduler.
 - Whenever a task of the same kind is processed, we are given that the task has a visibility timeout of n until it can be processed again.
 - If there are no tasks we can schedule at a given moment, this will result in idle time, increasing the time it takes to process the full batch (slower CPU)
 - We process one task per clock interval (recall the clock synchronizes the CPU) - you may have heard of overclocking used to describe strategies to reduce the clock interval to increase CPU efficiency. Usually however, the clock interval is generally the minimum time needed to process the slowest task or instruction, ensuring all other instructions can be processed within that time. Reducing it further may have unintended consequences such as some instructions not completing before the CPU moves to the next one. However, if such instructions are rare, overclocking can greatly reduce processing time, improving performance.
 - Given an array of tasks with possibly many of the same, we're asked to process them efficiently within a minimum number of clock intervals.

Strategy:
 - Greedy algorithm essentially: it's always in our best interest to proces the most frequent tasks first if possible so we aren't left with the overhead of processing them with idle time later if we don't prioritize them.
 - By prioritizing scheduling for the most frequent tasks, we ensure we reduce idle time as much as possible by fitting in between other tasks while there is a timeout on that task.
 - Hence, we'll use a max-heap (-1*freq) to have the most frequent tasks at at the top, and a queue to store (freq, timeout) until a task is ready to be on the heap again.
 - Use collections.Counter to gather the frequencies of each task (remember Counter works on characters too)
 - Go through the frequencies and drop them in a heap.
 - While both the heap and the queue aren't empty, we'll try to reheapify first if there's any tasks ready from the queue to be placed in the heap again, followed by popping off the most frequent current task on the heap to prioritize it. Keep track of each iteration (clock interval)
 - We'll then reduce the frequency by 1 since we've processed one more of that task, compute the timeout = clock_interval + n + 1 (add 1 so we wait n callouts correctly) and place to the back of the queue.
 - Return that clock_interval which should now be the minimum.
 - NOTE: knowing the actual task isn't important in this problem as long as we have the frequencies. 

 - Time complexity: O(nlogn): heap of n potential tasks if they all are unique. We'll have to do heappop an order of O(logn) times to schedule them, for total O(nlogn)
 - Memory complexity: O(n) for the heap of n elements.

In [ ]:
class Solution:
    def TaskScheduler(self, tasks: List[str], n: int) -> int:
        q = collections.deque()
        freqMap = collections.Counter(tasks)

        heap = []
        for num, freq in freqMap.items():
            heap.append(-1*freq)
        
        heapq.heapify(heap) #O(n)

        clock_interval = 0

        while heap or q:
            if q:
                #if the timeout is now less than clock_interval, (our curren time has exceeded the timeout) it's ready to go back in heap!
                if clock_interval >= q[0][1]:
                    tup = q.popleft()
                    heapq.heappush(heap, tup[0])
                
            if heap:
                freq = heapq.heappop(heap)
                #remember: freq here is negative so maintain a max heap.
                freq += 1
                #if freq == 0  now, we have no more of this task to process, so don't put it in the queue!
                if freq != 0:
                    timeout = clock_interval + n + 1
                    q.append((freq, timeout))
            
            clock_interval += 1
        return clock_interval